In [1]:
print('test')

test


# Imports

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import os
funcs = os.listdir('functions')

for func in funcs:
    if func.startswith('__'):
        continue
    func = func.replace('.py','')
    print(f'{func} installed')
    exec(f'from functions.{func} import *')

get_kaggle_data installed


In [104]:

import pandas as pd 

In [110]:
get_kaggle_data(competition="titanic")

files downloaded for titanic competition
   test.csv
   titanic.zip
   train.csv
   gender_submission.csv
located in /competition_data/
